In [18]:
import pandas as pd
from pathlib import Path
from bitcoin_risk.risk import prepare_and_compute_risk, canonical_columns, extend_dates, log_fits, btc_fit

# complete data-range
start = pd.to_datetime('2015-08-03')
end = pd.to_datetime('2024-11-30')
future = pd.to_datetime('2034-11-30')
last_date = '2019-11-30'

# result directory
current_dir = Path.cwd()
# path = current_dir / "eth" / last_date
path = current_dir / "docs" / "eth" / last_date
if not path.exists():
    path.mkdir(parents=True) 

df = prepare_and_compute_risk('data/INDEX_ETHUSD.csv', start, end, future, last_date)
# display(df)
from bitcoin_risk.plotter import create_figures
fig1, fig2, fig3, fig4 = create_figures(df, path)
fig1.show(); fig2.show(); fig3.show(); fig4.show()